In [97]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff

In [24]:
CFG = config.Config()

In [16]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,195.0,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,2820.0,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,118.0,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0


In [6]:
MODEL: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
CFG.prompt_classify_files.keys()

dict_keys(['interactivity_acknowledgement', 'political_ideology', 'political_negativity', 'rationality_background_info', 'rationality_external_evidence', 'rationality_reasoning', 'rationality_topic_relevance'])

In [8]:
list(CFG.prompt_classify_files.items())[0]

('interactivity_acknowledgement',
 PosixPath('data/prompts_classify/interactivity_acknowledgement.json'))

In [9]:
dataset["commentText"][:5]

0                                                  sad
1    That's a vicious insult!!! What did a box of r...
2    @colbertlateshow The question has always been ...
3    Goya Solidar.  So there are a few of us left. ...
4              hello hello \nNo-one else will hug him.
Name: commentText, dtype: object

In [35]:
for index, row in tqdm.tqdm(dataset["commentText"][:5].items()):
    print(index, row)

5it [00:00, 6962.66it/s]

0 sad
1 That's a vicious insult!!! What did a box of rocks ever do to you that you would slander it like that? I represent the coalition for mineral rights. Minerals have rights to.
2 @colbertlateshow The question has always been if he was compromised. We never knew if he was aware of the trump tow… https://t.co/J7oTDPZcM9
3 Goya Solidar.  So there are a few of us left.  Try reading the comments on both Fox News and The Daily Show clips.  The death of free and civil dialogue has never been painted so well.
4 hello hello 
No-one else will hug him.


In [56]:
#inspect GLLM reasoning to ensure valid processing of task and correct task interpretation:
            
predictions: typing.Dict[str, np.ndarray] = {}
for index, row in tqdm.tqdm(dataset["commentText"][:5].items()):
    try: 
        output = np.array(
            requests.post(
                'https://inf.cl.uni-trier.de/',
                json={
                    'model': MODEL,
                    'prompt': f"Does this comment acknowledge a previously posted user-comment or claim of another discussant? \nInstruction: By referring back to another comment, one recognizes the existence of a previous comment. Acknowledgment can be neutral or by explicitly endorsing another discussant's comment (e.g. 'I absolutely agree with what you're saying'). Keywords such as 'Yes; Yep; Exactly; I totally agree' may signal acknowledgement of a comment of another discussant. Acknowledgement can also be explicitly disagreeing with another discussant's comment or claim (e.g. '@RoniBox No, you're not right. Climate change is not real!'). \nA comment coded as YES for Acknowledgement is also likely to be coded as YES for Interaction. \nExample 1: '@tirtha simanta yeah but now someone deleted all of it' \nExample 2: 'Nope, that is not true' (is 1 under isReply) \nException 1: Code as NO if there is NOT a @username (or +1username, or username) NOR it is NOT a reply, because in most cases that means the comment is not referring to a commenter involved in the discussion. Whether this comment is a reply or not is indicated by 'isReply' ('1' means it is a reply) \nException 2: Code as NO if the comment is simply criticizing or cursing at another discussant (e.g. @Roompie Think properly, you fool), without explicitly saying that the writer (dis)agrees with another discussant. \\n\\n First explain your reasoning, then respond with the predicted class (0 or 1) of the request.\\n\\nText: {row}"                        
                    }).json()['response'])                  
        
                    
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")
        output = None
        
    predictions[index] = output

0it [00:00, ?it/s]

5it [00:24,  4.94s/it]


In [57]:
predictions


{0: array(' This comment does not acknowledge a previously posted user-comment or claim of another discussant. The text simply says "sad" which doesn\'t refer to any specific comment or user. Therefore, I would predict that this comment is coded as NO for Acknowledgement and also NO for Interaction since it doesn\'t interact with any other comments.',
       dtype='<U338'),
 1: array(' This comment does not appear to acknowledge a previously posted user-comment or claim of another discussant, as there is no explicit reference to or endorsement of another user\'s comment. The comment contains a playful tone, addressing "a box of rocks" and claiming to represent the "coalition for mineral rights," but it does not engage with any specific points raised by other users. Therefore, I would predict that this request should be coded as 0 (NO) for Acknowledgement.\n\nThat being said, without more context, it is possible that this comment is a reply to a specific user or thread of comments that 

apparently, it Llama3.1 expects the input text to mention the isReply-variable (which it does not), which might bias the results of the coding
mixtral however seems to ignore this part of the instruction
overall the reasoning provided does not indicate serious interpretation problems of the model with regards to the instructions

In [11]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items()
}

classifying rationality_reasoning: 100%|██████████| 3862/3862 [52:02<00:00,  1.24it/s]  


In [ ]:
predictions

In [12]:
dataset_w_pred = dataset.copy()

for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred = dataset_w_pred.join(preds)
    

No     2526
Yes    1332
Name: interactivity_acknowledgement, dtype: int64
------------------------------------------
neutral         1750
liberal         1325
conservative     784
Name: political_ideology, dtype: int64
------------------------------------------
No     3251
Yes     600
Name: rationality_external_evidence, dtype: int64
------------------------------------------
Yes    2555
No     1303
Name: rationality_topic_relevance, dtype: int64
------------------------------------------
political/negative        2560
not political/negative    1297
Name: political_negativity, dtype: int64
------------------------------------------
No     3158
Yes     700
Name: rationality_background_info, dtype: int64
------------------------------------------
No     2280
Yes    1576
Name: rationality_reasoning, dtype: int64
------------------------------------------


In [ ]:
dataset_w_pred.to_json(f'{CFG.report_dir}/publicsphere.prompt_classify.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred.to_parquet(f'{CFG.report_dir}/publicsphere.prompt_classify.parquet')
dataset_w_pred

In [4]:
MODELS: typing.Tuple[str, str] = [
    ("topics", "cardiffnlp/tweet-topic-21-multi"),
    ("emotions", "cardiffnlp/twitter-roberta-base-emotion-multilabel-latest"),
    ("sentiment", "cardiffnlp/twitter-roberta-base-sentiment-latest"),
    ("irony", "cardiffnlp/twitter-roberta-base-irony"),
    ("offensive",  "cardiffnlp/twitter-roberta-base-offensive"),
    ("hate", "cardiffnlp/twitter-roberta-base-hate-latest")
]

In [5]:
Cardiff_predictions: typing.Dict[str, pd.Series] = {}

In [6]:
for label, model in MODELS:
    classifier = src.HFClassify(slug=model)
    Cardiff_predictions[label] = pd.Series(classifier(dataset["commentText"], theta=0.4))

100%|██████████| 121/121 [07:06<00:00,  3.52s/it]
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 121/121 [07:04<00:00,  3.50s/it]


In [8]:
Cardiff_predictions

{'topics': 0                                                      {}
 1                                 {news_&_social_concern}
 2                                 {news_&_social_concern}
 3                                 {news_&_social_concern}
 4                                  {diaries_&_daily_life}
                               ...                        
 3857    {news_&_social_concern, business_&_entrepreneu...
 3858                              {news_&_social_concern}
 3859             {film_tv_&_video, news_&_social_concern}
 3860                              {news_&_social_concern}
 3861                              {news_&_social_concern}
 Length: 3862, dtype: object,
 'emotions': 0                  {pessimism, sadness}
 1                      {disgust, anger}
 2        {anticipation, anger, disgust}
 3                             {disgust}
 4                             {sadness}
                      ...               
 3857                   {disgust, anger}
 3858       

In [9]:
for label, series in Cardiff_predictions.items():
    display(label)
    display(series.value_counts())

'topics'

{news_&_social_concern}                                                                        2161
{diaries_&_daily_life}                                                                          240
{}                                                                                              208
{film_tv_&_video, news_&_social_concern}                                                        146
{diaries_&_daily_life, news_&_social_concern}                                                    97
                                                                                               ... 
{diaries_&_daily_life, news_&_social_concern, music}                                              1
{film_tv_&_video, arts_&_culture}                                                                 1
{news_&_social_concern, youth_&_student_life, sports, learning_&_educational}                     1
{film_tv_&_video, news_&_social_concern, business_&_entrepreneurs, celebrity_&_pop_culture}       1


'emotions'

{disgust, anger}                       1481
{disgust, anger, sadness}               278
{anticipation}                          263
{}                                      210
{joy, optimism}                         200
                                       ... 
{optimism, fear, sadness}                 1
{anticipation, fear, disgust}             1
{disgust, pessimism, fear, sadness}       1
{fear, surprise}                          1
{anticipation, joy, anger}                1
Name: count, Length: 86, dtype: int64

'sentiment'

...

{non_irony, irony}    1874
{non_irony}           1042
{irony}                946
Name: count, dtype: int64

'offensive'

{non-offensive}               2066
{offensive, non-offensive}    1158
{offensive}                    638
Name: count, dtype: int64

'hate'

{NOT-HATE}          3532
{HATE}               276
{HATE, NOT-HATE}      54
Name: count, dtype: int64

In [ ]:
dataset_w_pred = pd.read_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify.parquet')

for labels, preds in Cardiff_predictions.items():
    dataset_w_pred_2 = dataset_w_pred.join(preds.rename(labels))

dataset_w_pred_2

In [10]:
dataset_w_pred_2 = dataset_w_pred_2.drop(columns=['sentiment', 'offensive', 'topics', 'emotions', 'irony', 'hate'])

In [11]:
for labels, preds in Cardiff_predictions.items():
    dataset_w_pred_2 = dataset_w_pred_2.join(preds.rename(labels))

dataset_w_pred_2

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,rationality_topic_relevance,political_negativity,rationality_background_info,rationality_reasoning,topics,emotions,sentiment,irony,offensive,hate
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,No,not political/negative,No,No,{},"{pessimism, sadness}","{neutral, negative}","{non_irony, irony}","{offensive, non-offensive}",{NOT-HATE}
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,Yes,not political/negative,No,Yes,{news_&_social_concern},"{disgust, anger}","{neutral, negative}",{non_irony},"{offensive, non-offensive}",{NOT-HATE}
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,Yes,political/negative,No,No,{news_&_social_concern},"{anticipation, anger, disgust}","{neutral, negative}",{non_irony},{non-offensive},{NOT-HATE}
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,Yes,political/negative,Yes,Yes,{news_&_social_concern},{disgust},"{neutral, negative}",{irony},{non-offensive},{NOT-HATE}
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,No,not political/negative,No,No,{diaries_&_daily_life},{sadness},"{neutral, positive}","{non_irony, irony}",{non-offensive},{NOT-HATE}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,Yes,political/negative,No,Yes,"{news_&_social_concern, business_&_entrepreneu...","{disgust, anger}","{neutral, negative}",{non_irony},{non-offensive},{HATE}
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,Yes,political/negative,No,No,{news_&_social_concern},{anticipation},{neutral},{non_irony},{non-offensive},{NOT-HATE}
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,No,not political/negative,No,Yes,"{film_tv_&_video, news_&_social_concern}",{anticipation},"{neutral, negative}",{irony},{non-offensive},{NOT-HATE}
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,No,not political/negative,No,No,{news_&_social_concern},"{anticipation, optimism, trust}","{neutral, positive}",{non_irony},{non-offensive},{NOT-HATE}


In [14]:
# Select all columns that start with 'cardiff'
cardiff_columns = ['sentiment', 'offensive', 'topics', 'emotions', 'irony', 'hate']

# Convert each selected column to categorical type
for col in cardiff_columns:
    dataset_w_pred_2[col] = dataset_w_pred_2[col].astype('str').astype('category')

# Display the DataFrame to verify the changes
print(dataset_w_pred_2)

                StartDate         RecordedDate        IPAddress  Finished  \
0      5/30/2021 13:03:17   5/30/2021 13:04:17     62.194.51.29         1   
1     10/11/2021 10:34:05  10/11/2021 10:36:46  213.127.109.191         1   
2       9/9/2021 18:49:48    9/9/2021 18:51:32    213.127.110.0         1   
3       6/6/2021 16:12:46    6/6/2021 16:16:16   213.127.76.145         1   
4      6/13/2021 13:25:49   6/13/2021 13:27:28   213.127.82.232         1   
...                   ...                  ...              ...       ...   
3857   8/19/2021 14:50:13   8/19/2021 14:54:28     62.194.51.29         1   
3858   8/19/2021 15:10:27   8/19/2021 15:12:21     62.194.51.29         1   
3859   10/6/2021 16:08:39   10/6/2021 16:10:42  213.127.113.113         1   
3860  10/15/2021 18:30:04  10/15/2021 18:35:40  213.127.109.191         1   
3861  11/19/2021 17:49:17  11/19/2021 17:51:04  213.127.109.191         1   

      Coder                                                 ID   Mark_ID  \

In [18]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')

In [16]:
#anonymize:
dataset_w_pred_anon = dataset_w_pred_2.drop('commentText', axis=1)

In [17]:
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [75]:
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [12]:
dataset_w_pred_anon.loc[:, 'hate'].value_counts()

hate
{'NOT-HATE'}            3532
{'HATE'}                 276
{'HATE', 'NOT-HATE'}      54
Name: count, dtype: int64

In [76]:
#recode offensive to a dummy with non-offensive as 0 and offensive as 1:
dataset_w_pred_anon.loc[:, 'offensive_dum'] = dataset_w_pred_anon.loc[:, 'offensive'].map({"{'non-offensive'}": 0, "{'offensive', 'non-offensive'}": 1, "{'offensive'}": 1})
#recode hate into a dummy with non-hate as 0 and hate as 1:
dataset_w_pred_anon.loc[:, 'hate_dum'] = dataset_w_pred_anon.loc[:, 'hate'].map({"{'NOT-HATE'}": 0, "{'HATE'}": 1, "{'HATE', 'NOT-HATE'}": 1})

In [77]:
dataset_w_pred_anon.loc[:, 'cardiff_incivil'] = dataset_w_pred_anon.loc[:, ['offensive_dum', 'hate_dum']].max(axis=1)

In [78]:
print(dataset_w_pred_anon.loc[:, 'offensive'].value_counts())
print(dataset_w_pred_anon.loc[:, 'offensive_dum'].value_counts())
print(dataset_w_pred_anon.loc[:, 'hate'].value_counts())
print(dataset_w_pred_anon.loc[:, 'hate_dum'].value_counts())

offensive
{'non-offensive'}                 2066
{'offensive', 'non-offensive'}    1158
{'offensive'}                      638
Name: count, dtype: int64
offensive_dum
0    2066
1    1796
Name: count, dtype: int64
hate
{'NOT-HATE'}            3532
{'HATE'}                 276
{'HATE', 'NOT-HATE'}      54
Name: count, dtype: int64
hate_dum
0    3532
1     330
Name: count, dtype: int64


In [16]:
dimensions = pd.DataFrame([['Incivility',
               ['INCIVILITY_DUMMY','HATELIST_FOCUSED_DUMMY'],
               ['cardiff_incivil','hate_dum','offensive_dum'	
               ]]]  , 
            columns = ['dimension','labels','measures'])
dimensions

,dimension,labels,measures
0,Incivility,"[INCIVILITY_DUMMY, HATELIST_FOCUSED_DUMMY]","[cardiff_incivil, hate_dum, offensive_dum]"


In [17]:
print(dataset_w_pred_anon.hate_dum.dtypes)  # Verify the data types

int64


In [20]:
#calculate performance of cardiff's incivility classification:

# Reinitialize the DataFrame
performance = pd.DataFrame(columns=['Dimension', 'Label', 'Measures', 'Class', 'Precision', 'Recall', 'F1-score', 'support', 'Accuracy_overall'])

classif = ['0', '1', 'macro avg', 'weighted avg']

for index, dim in dimensions.iterrows():
    for label in dim['labels']:
        for measure in dim['measures']:
            try:
                data = dataset_w_pred_anon[dataset_w_pred_anon[measure].notna()]
                classfication = classification_report(data[label], data[measure], output_dict=True)
                for c in classif:
                    row = {
                        'Dimension': dim['dimension'],
                        'Label': label,
                        'Measures': measure,
                        'Class': c,
                        'Precision': classfication[c]['precision'],
                        'Recall': classfication[c]['recall'],
                        'F1-score': classfication[c]['f1-score'],
                        'support': classfication[c]['support'],
                        'Accuracy_overall': classfication['accuracy']
                    }
                    performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)
            except IndexError:
                pass

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_11660\3664728362.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)


In [21]:
performance

,Dimension,Label,Measures,Class,Precision,Recall,F1-score,support,Accuracy_overall
0,Incivility,INCIVILITY_DUMMY,cardiff_incivil,0,0.725886,0.746835,0.736212,1975.0,0.726308
1,Incivility,INCIVILITY_DUMMY,cardiff_incivil,1,0.726776,0.704822,0.715631,1887.0,0.726308
2,Incivility,INCIVILITY_DUMMY,cardiff_incivil,macro avg,0.726331,0.725829,0.725921,3862.0,0.726308
3,Incivility,INCIVILITY_DUMMY,cardiff_incivil,weighted avg,0.726321,0.726308,0.726156,3862.0,0.726308
4,Incivility,INCIVILITY_DUMMY,hate_dum,0,0.544734,0.974177,0.698747,1975.0,0.570430
5,Incivility,INCIVILITY_DUMMY,hate_dum,1,0.845455,0.147854,0.251691,1887.0,0.570430
6,Incivility,INCIVILITY_DUMMY,hate_dum,macro avg,0.695094,0.561015,0.475219,3862.0,0.570430
7,Incivility,INCIVILITY_DUMMY,hate_dum,weighted avg,0.691668,0.570430,0.480313,3862.0,0.570430
8,Incivility,INCIVILITY_DUMMY,offensive_dum,0,0.724105,0.757468,0.740411,1975.0,0.728379
9,Incivility,INCIVILITY_DUMMY,offensive_dum,1,0.733296,0.697933,0.715178,1887.0,0.728379


In [ ]:
#pretty good for incivility and on-par with ML model based on bert-base-uncased, fine-tuned on this dataset

In [19]:
#add a US political ideology prompt classification to the dataset, since it isn't a German one:
#load the dataset:
dataset_w_pred_2 = pd.read_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json')

In [36]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset_w_pred_2["commentText"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items() if label == 'political_ideology_US'
}

classifying political_ideology: 100%|██████████| 3862/3862 [27:05<00:00,  2.38it/s]


In [37]:
predictions #note that these are based on llama3.1:70b-instruct-q6_K

{'political_ideology_US': 0            neutral
 1       conservative
 2            liberal
 3            liberal
 4            neutral
             ...     
 3857         liberal
 3858         neutral
 3859         neutral
 3860         neutral
 3861         neutral
 Name: political_ideology, Length: 3862, dtype: object}

In [39]:
#join to the dataset:
for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred_2 = dataset_w_pred_2.join(preds)

political_ideology
neutral         2012
liberal         1075
conservative     775
Name: count, dtype: int64
------------------------------------------


In [43]:
#does political ideology differ from political ideology US?
#make a crosstab to see if there are differences:
pd.crosstab(dataset_w_pred_2.loc[:, 'political_ideology'], dataset_w_pred_2.loc[:, 'political_ideology_US'])

political_ideology_US,conservative,liberal,neutral
political_ideology,,,
conservative,649,29,106
liberal,75,977,273
neutral,50,68,1632


In [44]:
#strong overlap, but some differences, drop political ideology from the dataset:
dataset_w_pred_2 = dataset_w_pred_2.drop(columns=['political_ideology'])


In [45]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')

In [46]:
#anonymize:
dataset_w_pred_anon = dataset_w_pred_2.drop('commentText', axis=1)
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [101]:
#add liberal and conservative dummies based on political ideology US:
dataset_w_pred_anon.loc[:, 'political_conservative_US'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"liberal": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_opinion_US'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"liberal": 1, 'conservative': 1}).fillna(0).astype(int)

In [102]:
#print frequencies:
print(dataset_w_pred_anon.loc[:, 'political_conservative_US'].value_counts())
print(dataset_w_pred_anon.loc[:, 'political_liberal_US'].value_counts())
print(dataset_w_pred_anon.loc[:, 'political_opinion_US'].value_counts())


political_conservative_US
0    3087
1     775
Name: count, dtype: int64
political_liberal_US
0    2787
1    1075
Name: count, dtype: int64
political_opinion_US
0    2012
1    1850
Name: count, dtype: int64


In [53]:
dimensions = pd.DataFrame([['Diversity',
               ['HAS_OPINION_DUMMY','LIBERAL_DUMMY','CONSERVATIVE_DUMMY'],
               ['political_conservative_US', 'political_liberal_US', 'political_opinion_US'
               ]]]  , 
            columns = ['dimension','labels','measures'])
dimensions

,dimension,labels,measures
0,Diversity,"[HAS_OPINION_DUMMY, LIBERAL_DUMMY, CONSERVATIV...","[political_conservative_US, political_liberal_..."


In [54]:
#calculate performance of cardiff's incivility classification:

# Reinitialize the DataFrame
performance = pd.DataFrame(columns=['Dimension', 'Label', 'Measures', 'Class', 'Precision', 'Recall', 'F1-score', 'support', 'Accuracy_overall'])

classif = ['0', '1', 'macro avg', 'weighted avg']

for index, dim in dimensions.iterrows():
    for label in dim['labels']:
        for measure in dim['measures']:
            try:
                data = dataset_w_pred_anon[dataset_w_pred_anon[measure].notna()]
                classfication = classification_report(data[label], data[measure], output_dict=True)
                for c in classif:
                    row = {
                        'Dimension': dim['dimension'],
                        'Label': label,
                        'Measures': measure,
                        'Class': c,
                        'Precision': classfication[c]['precision'],
                        'Recall': classfication[c]['recall'],
                        'F1-score': classfication[c]['f1-score'],
                        'support': classfication[c]['support'],
                        'Accuracy_overall': classfication['accuracy']
                    }
                    performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)
            except IndexError:
                pass

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_4848\3664728362.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)


In [55]:
performance

,Dimension,Label,Measures,Class,Precision,Recall,F1-score,support,Accuracy_overall
0,Diversity,HAS_OPINION_DUMMY,political_conservative_US,0,0.572076,0.926062,0.707249,1907.0,0.621440
1,Diversity,HAS_OPINION_DUMMY,political_conservative_US,1,0.818065,0.324297,0.464469,1955.0,0.621440
2,Diversity,HAS_OPINION_DUMMY,political_conservative_US,macro avg,0.695070,0.625179,0.585859,3862.0,0.621440
3,Diversity,HAS_OPINION_DUMMY,political_conservative_US,weighted avg,0.696599,0.621440,0.584350,3862.0,0.621440
4,Diversity,HAS_OPINION_DUMMY,political_liberal_US,0,0.602440,0.880440,0.715381,1907.0,0.654065
5,Diversity,HAS_OPINION_DUMMY,political_liberal_US,1,0.787907,0.433248,0.559076,1955.0,0.654065
6,Diversity,HAS_OPINION_DUMMY,political_liberal_US,macro avg,0.695173,0.656844,0.637229,3862.0,0.654065
7,Diversity,HAS_OPINION_DUMMY,political_liberal_US,weighted avg,0.696326,0.654065,0.636257,3862.0,0.654065
8,Diversity,HAS_OPINION_DUMMY,political_opinion_US,0,0.764414,0.806502,0.784894,1907.0,0.781719
9,Diversity,HAS_OPINION_DUMMY,political_opinion_US,1,0.800541,0.757545,0.778449,1955.0,0.781719


In [ ]:
#very good performance for all political ideology classifications


In [88]:
#recode rationality items into dummies:
dataset_w_pred_anon.loc[:, 'rationality_reasoning_dum'] = dataset_w_pred_anon.loc[:, 'rationality_reasoning'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

In [84]:
#create a dummy for prompt coded rationality, with 1 if any of the following are coded as 'Yes': 'rationality_reasoning','rationality_background_info','rationality_external_evidence'
dataset_w_pred_anon.loc[:, 'rationality_prompt_dum'] = dataset_w_pred_anon.loc[:, ['rationality_reasoning_dum','rationality_background_info_dum','rationality_external_evidence_dum']].max(axis=1)

In [85]:
dataset_w_pred_anon.loc[:, ['rationality_reasoning','rationality_background_info','rationality_external_evidence', 'rationality_reasoning_dum','rationality_background_info_dum','rationality_external_evidence_dum','rationality_prompt_dum']]

,rationality_reasoning,rationality_background_info,rationality_external_evidence,rationality_reasoning_dum,rationality_background_info_dum,rationality_external_evidence_dum,rationality_prompt_dum
0,No,No,No,0.0,0.0,0.0,0.0
1,Yes,No,No,1.0,0.0,0.0,1.0
2,No,No,No,0.0,0.0,0.0,0.0
3,Yes,Yes,No,1.0,1.0,0.0,1.0
4,No,No,No,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3857,Yes,No,Yes,1.0,0.0,1.0,1.0
3858,No,No,No,0.0,0.0,0.0,0.0
3859,Yes,No,No,1.0,0.0,0.0,1.0
3860,No,No,No,0.0,0.0,0.0,0.0


In [94]:
dataset_w_pred_anon.loc[:, 'rationality_prompt_dum'] = dataset_w_pred_anon.loc[:, 'rationality_prompt_dum'].astype('int')

In [66]:
llm_human_column_pairs = [
    ("LIBERAL_DUMMY", "political_liberal_US"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US"),
    ("HAS_OPINION_DUMMY", "political_opinion_US"),
    ("RATIONALITY_DUMMY", "rationality_prompt_dum"),
   # ("TopicRelevance", "rationality_topic_relevance"),
]

In [103]:
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.93      0.85      0.89      3073
           1       0.56      0.77      0.65       789

    accuracy                           0.83      3862
   macro avg       0.75      0.81      0.77      3862
weighted avg       0.86      0.83      0.84      3862

cohen_kappa_score(LIBERAL_DUMMY): 0.5409748950370545
krippendorf(LIBERAL_DUMMY): 0.5375974087799733
---
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3271
           1       0.57      0.74      0.64       591

    accuracy                           0.87      3862
   macro avg       0.76      0.82      0.78      3862
weighted avg       0.89      0.87      0.88      3862

cohen_kappa_score(CONSERVATIVE_DUMMY): 0.5676841210411792
krippendorf(CONSERVATIVE_DUMMY): 0.566055054744786
---
              precis

In [ ]:
#although F1-score for rationality is decent, the cohen's kappa is very low, indicating poor agreement between human and LLM classification, probably due to low precision on positive class
#performance is on-par than supervised machine learning models in the public sphere paper, but less worse than finetuned bert-base-uncased model (0.72 macro avg precision, recall and f1)
#however krippendorf of manual coding for rationality items was in same range as LLM classification, indicating that the task is difficult to classify
#apparently mainly the reasoning/argumentation item is difficult to classify (manual K-alpha = 0.3; llm-human K-alpha = 0.05), as the other two items have a higher agreement between human and LLM classification

In [104]:
#save the dataset:
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')